In [1]:
# Boto3 session with credentials taken from environtment
import boto3
import os
import json
import botocore.exceptions

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())


session = boto3.Session(
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
    aws_session_token=os.environ["AWS_SESSION_TOKEN"],
)
kendra_client = session.client("kendra")
# translate_client = session.client("translate")
#lambda_client = session.client("lambda")
print(boto3.__version__)

# kendra_index = "630938f2-8264-4ca9-bcb5-5a1ad0025995"
#kendra_index = f"arn:aws:kendra:us-east-1:461004555540:index/{kendra_index}"

1.28.48


In [2]:
kendra_client.list_indices()['IndexConfigurationSummaryItems']

[{'Name': 'kendra-index',
  'Id': '9cfc368a-f4d6-4469-96bf-a6476b94ac31',
  'Edition': 'DEVELOPER_EDITION',
  'CreatedAt': datetime.datetime(2023, 9, 11, 20, 33, 57, 519000, tzinfo=tzlocal()),
  'UpdatedAt': datetime.datetime(2023, 9, 11, 20, 33, 57, 519000, tzinfo=tzlocal()),
  'Status': 'ACTIVE'}]

In [10]:
kendra_index = "9cfc368a-f4d6-4469-96bf-a6476b94ac31"

In [8]:
with open ("../kendra_constructs/template.to.compare.json") as f:
    template =  json.load(f)

In [13]:
template

{'connectionConfiguration': {'repositoryEndpointMetadata': {'s3SeedUrl': 's3://kendra-index-urlsbucket330222cc-uh1kqmon5xj7/s3_urls/connect_blogs_1.txt',
   'siteMapUrls': None,
   'seedUrlConnections': None,
   's3SiteMapUrl': None,
   'authentication': 'NoAuthentication'}},
 'enableIdentityCrawler': False,
 'syncMode': 'FORCED_FULL_CRAWL',
 'additionalProperties': {'inclusionFileIndexPatterns': [],
  'rateLimit': '300',
  'maxFileSize': '50',
  'crawlDepth': '1',
  'crawlAllDomain': False,
  'crawlSubDomain': False,
  'inclusionURLIndexPatterns': ['*.aws.amazon.com/blogs/contact-center/.*'],
  'exclusionFileIndexPatterns': [],
  'proxy': {},
  'exclusionURLCrawlPatterns': ['*./tag/.*'],
  'exclusionURLIndexPatterns': ['*./tag/.*'],
  'crawlAttachments': False,
  'honorRobots': True,
  'inclusionURLCrawlPatterns': ['*.aws.amazon.com/blogs/contact-center/.*'],
  'crawlDomainsOnly': True,
  'maxLinksPerUrl': '100'},
 'type': 'WEBCRAWLERV2',
 'version': '1.0.0',
 'repositoryConfiguration

In [11]:
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/kendra/client/create_data_source.html
try:
    res = kendra_client.create_data_source(
        Name="connect-blogs-100-v5",
        IndexId="9cfc368a-f4d6-4469-96bf-a6476b94ac31",
        Type="TEMPLATE",
        Configuration={"TemplateConfiguration": {"Template": template}},
        Description="Creado con Boto3 sdk",
        Schedule="",
        RoleArn="arn:aws:iam::320205374826:role/kendra-index-IKSRC7156E80-UBOWSCNEAXV9",
        LanguageCode="en"
    )
except botocore.exceptions.ClientError as error:
    # Put your error handling logic here
    print(error)

In [12]:
res['Id']

'5263e0fe-f0d0-4d9b-ba6f-9b0172db7aa3'

In [14]:
ds_id = "57a25b9f-e06c-4894-87cd-ea7b84bd63a5"
ds = kendra_client.describe_data_source(IndexId=kendra_index, Id=ds_id)

In [15]:
ds

{'Id': '57a25b9f-e06c-4894-87cd-ea7b84bd63a5',
 'IndexId': '9cfc368a-f4d6-4469-96bf-a6476b94ac31',
 'Name': 'connect-blogs-100-v4',
 'Type': 'TEMPLATE',
 'Configuration': {'TemplateConfiguration': {'Template': {'connectionConfiguration': {'repositoryEndpointMetadata': {'s3SeedUrl': 's3://kendra-index-urlsbucket330222cc-uh1kqmon5xj7/s3_urls/connect_blogs_1.txt',
      'siteMapUrls': None,
      'seedUrlConnections': None,
      's3SiteMapUrl': None,
      'authentication': 'NoAuthentication'}},
    'enableIdentityCrawler': False,
    'syncMode': 'FORCED_FULL_CRAWL',
    'additionalProperties': {'inclusionFileIndexPatterns': [],
     'rateLimit': '300',
     'maxFileSize': '50',
     'crawlDepth': '1',
     'crawlAllDomain': False,
     'crawlSubDomain': False,
     'inclusionURLIndexPatterns': ['*.aws.amazon.com/blogs/contact-center/.*'],
     'exclusionFileIndexPatterns': [],
     'proxy': {},
     'exclusionURLCrawlPatterns': ['*./tag/.*'],
     'exclusionURLIndexPatterns': ['*./tag/.

In [2]:
first, second = "123|456".split("|")

In [4]:
first

'123'

In [2]:
def QueryKendra(index_id, query):
    response = kendra_client.query(QueryText=query, IndexId=index_id)
    return response


def RetrieveKendra(index_id, query, domain=None):
    kwargs = dict(QueryText=query, IndexId=index_id)
    if domain:
        kwargs["AttributeFilter"] = {
                "EqualsTo": {
                    "Key": "domain",
                    "Value": {
                        "StringValue": domain,
                    },
                }
            }
    response = kendra_client.retrieve(**kwargs)
    return response

In [6]:
translate_response = translate_client.translate_text(
    Text= "qué es amazon connect?",
    SourceLanguageCode="auto", TargetLanguageCode="en"
)

question =  translate_response['TranslatedText']
response_retrieve = RetrieveKendra(kendra_index, question, domain='Connect')

ValidationException: An error occurred (ValidationException) when calling the Retrieve operation: Invalid resource type passed in ARN index/arn:aws:kendra:us-east-1:461004555540:index/630938f2-8264-4ca9-bcb5-5a1ad0025995

In [4]:
response_retrieve['ResultItems']

[{'Id': '36e89d96-90cb-47d5-8fac-74889e4c21f2-3cc652fa-913a-4915-82e8-8b81acaab768',
  'DocumentId': 'https://docs.aws.amazon.com/connect/latest/adminguide/connect-slr.html',
  'DocumentTitle': 'Use service-linked roles for Amazon Connect - Amazon Connect',
  'Content': 'Contact Us English AWS Documentation Amazon Connect Administrator Guide Feedback Preferences Use service-linked roles for Amazon Connect PDF RSS What are service-linked roles (SLR) and why are they important? Amazon Connect uses AWS Identity and Access Management (IAM) service-linked roles. A service-linked role is a unique type of IAM role that is linked directly to an Amazon Connect instance. Service-linked roles are predefined by Amazon Connect and include all the permissions that Amazon Connect requires to call other AWS services on your behalf. You need to enable service-linked roles so you can use new features in Amazon Connect, such as tagging support, the new user interface in User management and Routing profil

In [16]:
response_retrieve

{'QueryId': '60bf9d10-32de-41f5-9036-4e3548836ccd',
 'ResultItems': [{'Id': '60bf9d10-32de-41f5-9036-4e3548836ccd-7f287ef6-c064-4d36-ae1a-37015222c2f7',
   'DocumentId': 'https://docs.aws.amazon.com/connect/latest/adminguide/connect-slr.html',
   'DocumentTitle': 'Use service-linked roles for Amazon Connect - Amazon Connect',
   'Content': 'Contact Us English AWS Documentation Amazon Connect Administrator Guide Feedback Preferences Use service-linked roles for Amazon Connect PDF RSS What are service-linked roles (SLR) and why are they important? Amazon Connect uses AWS Identity and Access Management (IAM) service-linked roles. A service-linked role is a unique type of IAM role that is linked directly to an Amazon Connect instance. Service-linked roles are predefined by Amazon Connect and include all the permissions that Amazon Connect requires to call other AWS services on your behalf. You need to enable service-linked roles so you can use new features in Amazon Connect, such as taggin

In [17]:
def get_full_body(attr_list):
    full_body = None
    for attr in attr_list:
        if attr['Key'] == 'fullbody':
            full_body = attr['Value']['StringValue']
            full_body = full_body.replace("Contact Us\nEnglish\nAWS\nDocumentation\nAmazon Connect\nAdministrator Guide\nFeedback\nPreferences\n", "")
            full_body = full_body.replace("PDF\nRSS", "")
            full_body = full_body.split("\nRELATED POSTS\n")[-1]
            full_body = full_body.split("| Permalink |  Share\n")[-1]
            full_body = full_body.split("TAGS:")[0]
            full_body = full_body.split("\nDid this page help you?")[0]
            full_body = full_body.replace("}","").replace("{","").replace('"', "'")
    return full_body

documents = []
for rs in response_retrieve['ResultItems']:
    full_body = get_full_body(rs['DocumentAttributes'])
    document = {"DocumentURI": rs['DocumentURI'], "Content": full_body}
    documents.append(document)

unique_url = []
unique_documents = []
for doc in documents:
    if doc['DocumentURI'] in unique_url: continue
    else: 
        unique_url.append(doc['DocumentURI'])
        unique_documents.append(doc)

In [18]:
unique_documents

[{'DocumentURI': 'https://docs.aws.amazon.com/connect/latest/adminguide/connect-slr.html',
  'Content': "Use service-linked roles for Amazon Connect\n\nWhat are service-linked roles (SLR) and why are they important?\nAmazon Connect uses AWS Identity and Access Management (IAM) service-linked roles. A service-linked role is a unique type of IAM role that is linked directly to an Amazon Connect instance.\nService-linked roles are predefined by Amazon Connect and include all the permissions that Amazon Connect requires to call other AWS services on your behalf.\nYou need to enable service-linked roles so you can use new features in Amazon Connect, such as tagging support, the new user interface in User management and Routing profiles, and queues with CloudTrail support.\nFor information about other services that support service-linked roles, see AWS services that work with IAM and look for the services that have Yes in the Service-Linked Role column. Choose a Yes with a link to view the s

In [20]:
articles = ""
for i in range(3):
    ud = unique_documents[i]
    articles += f"Article {i}:\n\n{ud['Content']}"

len(articles.split(" "))

2581

In [21]:
from botocore.exceptions import ClientError

def call_lambda(function_arn_or_name, event):
    try:
        print(f"Intentando invocar :{function_arn_or_name}")
        response = lambda_client.invoke(
            FunctionName=function_arn_or_name,
            InvocationType="RequestResponse",
            Payload=json.dumps(event),
        )
        print ("FUCTION_RESPONSE")
        function_response = response["Payload"].read().decode("utf-8")
        print(function_response) 

        json_response = json.loads(function_response)
        print(f"\nRespuesta:{json_response}")
        json_body = json.loads(json_response["body"])
        return json_body

    except ClientError as e:
        err = e.response
        error = err
        print(err.get("Error", {}).get("Code"))
        return "Un error invocando Lambda"


In [26]:
def build_bedrock_payload(text, question):
    prompt = f"{text}\n\nBased on the above articles, answer briefly and concisely the following question:\n\n{question}"
    prompt += "\n\nGet your answer only from the articles, whitout making up new information."
    print(prompt)

    payload = {
        "body": {
            "prompt": f"Human: {prompt} Assistant:",
            "max_tokens_to_sample": 500,
            "temperature": 0.7,
            "top_k": 100,
            "top_p": 0.9,
            "stop_sequences": ["\n\nHuman"],
        },
        "modelId": "anthropic.claude-v2",
        "accept": "*/*",
        "contentType": "application/json",
    }
    print ("PAYLOAD:")
    print (payload)
    return payload

In [27]:
bedrock_payload =  build_bedrock_payload(articles, question)

Article 0:

Use service-linked roles for Amazon Connect

What are service-linked roles (SLR) and why are they important?
Amazon Connect uses AWS Identity and Access Management (IAM) service-linked roles. A service-linked role is a unique type of IAM role that is linked directly to an Amazon Connect instance.
Service-linked roles are predefined by Amazon Connect and include all the permissions that Amazon Connect requires to call other AWS services on your behalf.
You need to enable service-linked roles so you can use new features in Amazon Connect, such as tagging support, the new user interface in User management and Routing profiles, and queues with CloudTrail support.
For information about other services that support service-linked roles, see AWS services that work with IAM and look for the services that have Yes in the Service-Linked Role column. Choose a Yes with a link to view the service-linked role documentation for that service.
Service-linked role permissions for Amazon Conne

In [28]:
return_value = call_lambda("arn:aws:lambda:us-east-1:147654516002:function:BedrockLambda-FCallBedrock4BAEA876-A3tVXbSXt0qJ", bedrock_payload)

Intentando invocar :arn:aws:lambda:us-east-1:147654516002:function:BedrockLambda-FCallBedrock4BAEA876-A3tVXbSXt0qJ
FUCTION_RESPONSE
{"statusCode": 200, "headers": {"Content-Type": "text/html;charset=UTF-8", "charset": "UTF-8", "Access-Control-Allow-Origin": "*"}, "body": "{\"event\": {\"body\": {\"prompt\": \"Human: Article 0:\\n\\nUse service-linked roles for Amazon Connect\\n\\nWhat are service-linked roles (SLR) and why are they important?\\nAmazon Connect uses AWS Identity and Access Management (IAM) service-linked roles. A service-linked role is a unique type of IAM role that is linked directly to an Amazon Connect instance.\\nService-linked roles are predefined by Amazon Connect and include all the permissions that Amazon Connect requires to call other AWS services on your behalf.\\nYou need to enable service-linked roles so you can use new features in Amazon Connect, such as tagging support, the new user interface in User management and Routing profiles, and queues with CloudTra

In [29]:
print(return_value['response']['body']['completion'])

 Amazon Connect is a cloud-based contact center service from AWS that enables companies to provide customer service and support through voice, chat, and task automation. Key features include:

- Cloud-based contact center that can scale to support large call volumes.

- Integration with AWS services like Amazon Lex for chatbots and Amazon Polly for text-to-speech. 

- Omnichannel support for voice, chat, and task automation.

- APIs and SDKs for customization and integration with other systems.

- Pay-per-use pricing model with no upfront fees.

- Service-linked roles in IAM for access management. 

- Support for identity-based policies, resource tags, and temporary credentials.

The service allows companies to set up, operate, and scale customer engagement centers in the cloud. It works with other AWS tools to provide an end-to-end contact center platform.


In [ ]:
https://docs.aws.amazon.com/lexv2/latest/dg/contact-center.html
https://docs.aws.amazon.com/lexv2/latest/dg/bot-templates/contact-center.html